## Experiments

Let's start playing around with our data

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import tqdm
from sklearn.model_selection import train_test_split
from datasets import load_dataset,Dataset,DatasetDict
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification,AutoTokenizer,Trainer,TrainingArguments,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2024-04-14 22:53:23.797661: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 22:53:25.278915: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Read in training data
This dataset contains 100 annotated terms of service contracts, each row represents a sentence, which carries on it a label. The label corresponds to a different type of potential unfairness, as defined by the authors of CLAUDETTE, the previous paper from which this dataset came from. 

In [2]:
df = pd.read_csv('../data/dataset.csv')
df.head()

,Unnamed: 0,A,CH,CR,J,LAW,LTD,PINC,TER,USE,document,document_ID,label,text,TER_targets,LTD_targets,A_targets,CH_targets,CR_targets
0,0,0,0,0,0,0,0,0,0,0,Mozilla,0,0,websites & communications terms of use,NaN,NaN,NaN,NaN,NaN
1,1,0,0,0,0,0,0,0,0,0,Mozilla,0,0,please read the terms of this entire document ...,NaN,NaN,NaN,NaN,NaN
2,2,0,0,0,0,0,0,0,0,1,Mozilla,0,1,by accessing or signing up to receive communic...,NaN,NaN,NaN,NaN,NaN
3,3,0,0,0,0,0,0,0,0,0,Mozilla,0,0,our websites include multiple domains such as ...,NaN,NaN,NaN,NaN,NaN
4,4,0,0,0,0,0,0,0,0,0,Mozilla,0,0,you may also recognize our websites by nicknam...,NaN,NaN,NaN,NaN,NaN


Now let's load in a pretrained huggingface BERT model  (https://huggingface.co/pile-of-law/legalbert-large-1.7M-2) to get the word embeddings of each sentence

In [3]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model = BertModel.from_pretrained('pile-of-law/legalbert-large-1.7M-2')
model.to(device)
text = "This is a test"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input.to(device))
weights = model.get_input_embeddings()

In [4]:
output.pooler_output.shape
output.pooler_output[0].shape

torch.Size([1024])

# Oversample the training data
Since our classifier isn't returning any positive predictions, the problem could be that it isn't seeing enough examples of sentences that are potentially exploitative. Let's fix that

In [5]:
from imblearn.over_sampling import RandomOverSampler
x, y = df['text'], df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
ros = RandomOverSampler(random_state=42,sampling_strategy=.5)
x_train_res, y_train_res = ros.fit_resample(pd.DataFrame(x_train), pd.DataFrame(y_train))

In [6]:
train = pd.merge(x_train,y_train,left_index=True,right_index=True)
train_os_df = pd.merge(x_train_res,y_train_res,left_index=True,right_index=True)
test_df = pd.merge(x_test,y_test,left_index=True,right_index=True)
train_os_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [7]:
train = Dataset.from_pandas(train)
train_os = Dataset.from_pandas(train_os_df.sample(frac = .5))
test = Dataset.from_pandas(test_df)
test

Dataset({
    features: ['index', 'text', 'label'],
    num_rows: 4084
})

In [8]:
train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 16333
})

In [9]:
train_os

Dataset({
    features: ['index', 'text', 'label', '__index_level_0__'],
    num_rows: 10952
})

Split dataset into train, test and validation

In [10]:
test_valid = test.train_test_split(test_size=0.5,seed=42)

dataset_dict = DatasetDict({
    'train':train,
    'train_os':train_os,
    'test':test_valid['test'],
    'validation':test_valid['train']
    })
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 16333
    })
    train_os: Dataset({
        features: ['index', 'text', 'label', '__index_level_0__'],
        num_rows: 10952
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 2042
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 2042
    })
})

In [11]:
def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=1024)

In [12]:
tokenized_df = dataset_dict.map(tokenize, batched=True)

Map:   0%|          | 0/16333 [00:00<?, ? examples/s]

Map:   0%|          | 0/10952 [00:00<?, ? examples/s]

Map:   0%|          | 0/2042 [00:00<?, ? examples/s]

Map:   0%|          | 0/2042 [00:00<?, ? examples/s]

In [13]:
tokenized_df

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16333
    })
    train_os: Dataset({
        features: ['index', 'text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10952
    })
    test: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2042
    })
    validation: Dataset({
        features: ['index', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2042
    })
})

In [14]:
tokenized_df.set_format('torch', columns=['input_ids', 'attention_mask',"token_type_ids",'label'])
#data collator forms a batch with padding for the length of the longest input. We do this so we don't need to repeat unnecessary computations on a global maximum length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Now let's construct a custom classifier classifier to classify sentences as potentially unfair or not 

In [15]:
class Classifier_v1(nn.Module):
    def __init__(self, model_name):
        super(Classifier_v1,self).__init__()
        self.model = AutoModel.from_pretrained(model_name,config = AutoConfig.from_pretrained(model_name,
                                                                                              output_attention = True,
                                                                                              output_hidden_state = True))
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.hidden = nn.Linear(self.model.config.hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
    
    def forward(self, input = None, attention_mask = None, token_type_ids = None,input_ids = None, labels = None):
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids)
        pooler_output = outputs.pooler_output[0]
        output = self.hidden(pooler_output)
        logit = self.sigmoid(output)
        return logit

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_df['train'], shuffle=True, batch_size=1, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_df['test'], shuffle=True, batch_size=1,collate_fn=data_collator
)
train_os_dataloader = DataLoader(
    tokenized_df['train_os'], shuffle=True, batch_size=1, collate_fn=data_collator
)

## Training Loop

In [17]:
torch.cuda.empty_cache()

In [18]:
from transformers import get_scheduler
from torch.optim import Adam, SGD

model_v2 = Classifier_v1('pile-of-law/legalbert-large-1.7M-2').to(device)
loss_fn = nn.BCELoss()
optimizer = Adam(model_v2.parameters(),lr =.0001)
epochs = 5

In [19]:
training_steps = epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=training_steps
)

# Full Loop

In [22]:
def train(model,train_dataloader,epochs,loss_fn,optimizer,lr_scheduler,testing=False):
    for epoch in range(epochs):
        model.train()
        for batch in tqdm.tqdm(train_dataloader):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label = batch['labels'].to(torch.float32).to(device)
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            loss = loss_fn(output,label)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
        preds = []
        labels = []
        for i,batch in tqdm.tqdm(enumerate(test_dataloader)):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            label = batch['labels'].to(torch.float32).to(device)
            with torch.no_grad():
                output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
            loss = loss_fn(output,label)
            pred = torch.round(output)
            preds.append(pred.detach().cpu().numpy())
            labels.append(batch["labels"].detach().cpu().numpy())
        print(f"Epoch: {epoch + 1} | Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {loss}")

In [27]:
def test(model,test_dataloader,loss_fn):
    model.eval()
    preds = []
    labels = []
    for i,batch in tqdm.tqdm(enumerate(test_dataloader)):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label = batch['labels'].to(torch.float32).to(device)
        with torch.no_grad():
            output = model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        loss = loss_fn(output,label)
        pred = torch.round(output)
        preds.append(pred.detach().cpu().numpy())
        labels.append(batch["labels"].detach().cpu().numpy())
    print(f"Accuracy: {accuracy_score(labels,preds)} | Precision: {precision_score(labels,preds)} | Recall: {recall_score(labels,preds)} | F1: {f1_score(labels,preds)} | Loss: {loss}")

In [24]:
train(model_v2,train_os_dataloader,3,loss_fn,optimizer,lr_scheduler)

100%|██████████| 10952/10952 [16:16<00:00, 11.21it/s] 
2042it [02:54, 11.72it/s]


Epoch: 1 | Accuracy: 0.8555337904015671 | Precision: 0.369281045751634 | Recall: 0.5255813953488372 | F1: 0.43378119001919385 | Loss: 0.42886269092559814


100%|██████████| 10952/10952 [16:17<00:00, 11.20it/s]
2042it [03:25,  9.94it/s]


Epoch: 2 | Accuracy: 0.8501469147894222 | Precision: 0.34983498349834985 | Recall: 0.4930232558139535 | F1: 0.4092664092664093 | Loss: 0.34930482506752014


100%|██████████| 10952/10952 [15:49<00:00, 11.53it/s]
2042it [02:50, 12.01it/s]


Epoch: 3 | Accuracy: 0.8672869735553379 | Precision: 0.39705882352941174 | Recall: 0.5023255813953489 | F1: 0.44353182751540043 | Loss: 0.44554612040519714


In [26]:
torch.save(model_v2.state_dict(),'../models/model_v2.pth')

In [ ]:
val_dataloader = DataLoader(
    tokenized_df['validation'], shuffle=True, batch_size=1,collate_fn=data_collator
)
test(model_v2,test_dataloader,loss_fn)

## Post Training Evaluation

It looks like our model is simply labeling everything as not exploitative, which would explain why our f1 score is